In [ ]:
import pandas as pd 
import os
import nltk
from nltk.tokenize import word_tokenize 
from nltk import Tree
from nltk.draw.tree import TreeView
import os
from spacy import displacy
import spacy
from collections import Counter
import en_core_web_sm
from keras.preprocessing.text import text_to_word_sequence
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet


nltk.download('wordnet')
nlp = en_core_web_sm.load()
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
!pip install benepar
!pip install cython numpy
!pip install Keras


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


STIAHNUTIE TWITTER DAT


In [ ]:
import tweepy as tw
api_key= 'Xn4lLpcmofHaZBd7NvgyNTSaE'
api_secret= 'pPnlRPfbSA1rwgNM7L6NL1GIyOaruWHYRpaK4v9GmnomMtAaLp'
access_token= '1387459078046298115-1n3EyAk1rNf8mCfkubpdIvQ3FusIuh'
access_token_secret= 'TiMz3h1SN30lyGxEqeHozrSli1Q7WDi2mWwD5CFTNb8y6'

auth = tw.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)
words2search = "#economic -filter:retweets" #+ coronavirus
date = "2019-11-16"


@ArnoSnyman4 @_TshegoM Don’t drive BMW and manage my finances well am Black n proud.Tried to buy a car Absa decline… https://t.co/BKjiUGLlpF
RT @ajaymaken: My tributes to Dr A.K.Walia at the prayer meet-Underlined Dr Walia's contributions to Delhi as its Finance,Health &amp; Urban De…
@ArnoSnyman4 @_TshegoM Don’t drive BMW and manage my finances well am Black n proud.Tried to buy a car Absa decline… https://t.co/BKjiUGLlpF


In [ ]:
import csv
import time
import hashlib

last_id = None
result = True
all_tweets = []
all_hashes = []

num_of_requests = 0

while result:
  result = api.search(q=words2search, count=500, tweet_mode='extended', max_id=last_id)
  
  with open('new_tweets.txt', 'a') as f:
    for res in result:
      hash = hashlib.md5(res.full_text.encode('utf-8'))
      if hash in all_hashes:
        continue
      all_hashes.append(hash)
      all_tweets.append(res.full_text)
      f.write(res.full_text)
      f.write('\n---TWEET-BREAK---\n')

  f.close()

  num_of_requests += 1
  last_id = result[-1]._json['id'] - 1
  print('Iteration: {it} Finished iteration with last_id: {i} with total tweets of: {total}'.format(it=num_of_requests, i=last_id, total=len(all_tweets)))


In [ ]:
import re
def remove_url(txt):
  txt = txt.replace('\n', ' ').replace('\r', ' ')
  return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

def remove_numbers(txt):
  pattern = r'[0-9]'
  # Match all digits in the string and replace them by empty string
  return re.sub(pattern, '', txt)

In [ ]:
with open('new_tweets.txt', 'r') as f:
  content = f.read()
  content_list = content.split("\n---TWEET-BREAK---\n")
  data = pd.DataFrame(content_list,columns =['Tweets'])

In [ ]:
data['Tweets'] = data['Tweets'].apply(lambda row: remove_url(row))
data['Tweets'] = data['Tweets'].apply(lambda row: remove_numbers(row))


In [ ]:
data = data.drop_duplicates(subset=['Tweets'])
data.replace("", float("NaN"), inplace=True)
data.dropna(subset = ["Tweets"],inplace=True)
data.reset_index(drop=True)

In [ ]:
data.to_csv('tweetsdataset.csv')
!cp tweetsdataset.csv "/content/drive/My Drive/"

In [ ]:
data = pd.read_csv('/content/drive/My Drive/tweetsdataset.csv')

In [ ]:
class Sentence:
  def __init__(self):
    self.words = []
    self.wholeSentence = None
    self.tree = None
    self.NerTag = None

  def appendWords(self, word): 
    self.words.append(word) 

class Word:
  def __init__(self):
    self.word = None
    self.lemma = None
    self.synsetName = None
    self.synsetDefinition = None
    self.posTag = None
    self.grammarTag = None
    self.synonyma = []
    self.antonyma = None
    self.negation = None


In [ ]:

from nltk.corpus import stopwords 
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:


def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def get_first_sense(word, pos=None):
    if pos:
        synsets = wordnet.synsets(word,pos)
    else:
        synsets = wordnet.synsets(word)
    if not synsets:
      return None
    else:
      return synsets[0]

data['Postag-Lemmatized Sentence'] = None
data['Pos-Lem Sen without tags'] = None
listofSentences = []
for index,row in data.iterrows():
  #inicializácia objektov
  lm = WordNetLemmatizer()
  sentenceObject = Sentence()

  newEditedSentence = ""
  taggedlematizedsentence = []

  stop_words = stopwords.words('english')

  sentence = row['Tweets']
  print(index)
  tokenized_sentence = text_to_word_sequence(sentence)
  filtered_sentence = [w for w in tokenized_sentence if not w in stop_words] 
  tagged_sentence = nltk.pos_tag(filtered_sentence)

  for token in tagged_sentence:
    wordObject = Word()
    best_synset = None
    postag = get_wordnet_pos(token[1])
    wordObject.word = token[0]
    if postag is None or postag is '': # not supply tag in case of None
      lemma = lm.lemmatize(token[0])
      taggedlematizedsentence.append((lemma,token[1]))
      newEditedSentence+=lemma + " "
      best_synset = get_first_sense(lemma)
      wordObject.lemma = lemma
      wordObject.posTag = token[1]
    else:
      lemma = lm.lemmatize(token[0], postag) 
      taggedlematizedsentence.append((lemma,token[1])) # vráti sa mi lemma a pridam pôvodný POS-tag, nie ten ktorý mám z Wordnetu 
      best_synset = get_first_sense(lemma,postag)

      newEditedSentence+=lemma + " "
      wordObject.lemma = lemma
      wordObject.posTag = token[1]

    if best_synset is None:
      #print("Slovo nema viac významov") 
      wordObject.synsetName = None
      wordObject.synsetDefinition = None
      #TODO negation handling   
    else:
      wordObject.synsetName = best_synset.name()
      wordObject.synsetDefinition = best_synset.definition()
      for synlemma in best_synset.lemmas():
        wordObject.synonyma.append(synlemma.name())
        if synlemma.antonyms():
          wordObject.antonyma = synlemma.antonyms()[0].name()
          #print(wordObject.lemma,synlemma.antonyms()[0].name())


    sentenceObject.appendWords(wordObject)
  sentenceObject.wholeSentence = newEditedSentence
  data.at[index,'Postag-Lemmatized Sentence'] = taggedlematizedsentence
  data.at[index,'Pos-Lem Sen without tags'] = newEditedSentence
  data.at[index,'Sentence-word object'] = sentenceObject
  listofSentences.append(sentenceObject)

In [ ]:
import pickle 
filehandler = open("/content/drive/My Drive/Test_Twitter_Sentences", 'wb') 
pickle.dump(listofSentences, filehandler, pickle.HIGHEST_PROTOCOL)
filehandler.close()